In [ ]:
import pandas as pd
import os
import caselawnet
import sqlalchemy
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
filepath =  '/media/sf_VBox_Shared/CaseLaw/2018-01-29-lido/derived'

In [ ]:
cases_links = pd.read_csv(os.path.join(filepath, 'case_to_case_lx_links.csv'))

In [ ]:
cases_links.head()

In [ ]:
case_lido_ids = list(set(cases_links['source']).union(set(cases_links['target'])))
case_ecli = [s.split('/')[-1] for s in case_lido_ids]

In [ ]:
cases_df = pd.DataFrame({'lido_id': case_lido_ids, 'ecli': case_ecli})
cases_df.head()

In [ ]:
meta_df = pd.DataFrame([e.split(':')[1:4] for e in case_ecli], columns=['country', 'court', 'year'])
meta_df.head()

In [ ]:
cases_df = pd.concat([cases_df, meta_df], axis=1)

In [ ]:
cases_df.head()

In [ ]:
cases_df.country.value_counts()

In [ ]:
cases_df.court.value_counts()[:20]

In [ ]:
plt.figure(figsize=(20, 10))
cases_df.court.value_counts().plot(kind='bar')

In [ ]:
cases_df.year.value_counts(sort=False).sort_index().plot()

In [ ]:
cases_df.year.value_counts(sort=False).sort_index().cumsum().plot()

In [ ]:
cases_df.to_csv(os.path.join(filepath, 'case_nodes_simple.csv'), index=False)

## From database

In [ ]:
import zipfile

In [ ]:
rootpath = '/media/sf_VBox_Shared/CaseLaw/OpenDataUitspraken-maart/'

In [ ]:
dbpath = 'sqlite:////media/sf_VBox_Shared/CaseLaw/caselaw2018.db'
session = caselawnet.dbutils.get_session(adress=dbpath)

In [ ]:
# Fill database with necessary nodes
for dir0 in os.listdir(rootpath):
    print("Processing directory", dir0)
    dir0 = os.path.join(rootpath, dir0)
    if os.path.isdir(dir0):
        for dir1 in os.listdir(dir0):
            dir1 = os.path.join(dir0, dir1)
            if zipfile.is_zipfile(dir1):
                zf = zipfile.ZipFile(dir1, 'r')
                for n in zf.namelist():
                    ecli = n.split('.')[0].replace('_', ':')
                    if ecli in case_ecli:
                        caselawnet.dbutils.file_to_db(n, session, zf)

In [ ]:
caselawnet.enrich_eclis(['ECLI:NL:PHR:1984:AC8323'], rootpath='/media/sf_VBox_Shared/CaseLaw/OpenDataUitspraken/', db_session=session)

In [ ]:
session.execute('select max(date) from cases').fetchall()